In [1]:
import pandas as pd


In [ ]:
"""data preprocesing revies
1--> remove puntuation
2--> lower
3--> remove $%^&
4__> remove stop wrods like hotel , hotels . nyc, 



"""

In [4]:
# only upload two features
df_1 = pd.read_csv('five_hotels.csv', usecols = ['reviews', 'review_rating'])
df_2 = pd.read_csv('last_hotels.csv', usecols = ['reviews', 'review_rating'])
df = pd.concat([df_1, df_2])
# reseting index
df = df.reset_index(drop=True)

In [5]:
df.head()

,reviews,review_rating
0,Let me start off by saying that I understand t...,10
1,We had yet another wonderful visit at this hot...,50
2,It was one of the best hotels I have ever stay...,50
3,This hotel is just fantastic. The location is...,50
4,Amazing park views from this hotel : ) Nicole ...,50
